In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


In [95]:
def extract_relation_names(json_data):
    relation_names = set()
    
  # Using set to ensure unique values
    for item in json_data:
        
        
        extract_relation_names_from_plan(item, relation_names)
    return relation_names

def extract_relation_names_from_plan(plan, relation_names):
    relation_name = plan.get("Relation Name")
    if relation_name:
        relation_names.add(relation_name)
    if "Plans" in plan:
        for subplan in plan["Plans"]:
            extract_relation_names_from_plan(subplan, relation_names)


In [96]:
def extract_filters(data):
    filters = []

    if isinstance(data, dict):
        for key, value in data.items():
            if key == "Filter":
                filters.append(value)
            elif isinstance(value, (dict, list)):
                filters.extend(extract_filters(value))

    elif isinstance(data, list):
        for item in data:
            filters.extend(extract_filters(item))

    return filters

In [97]:
def extract_actual_rows(data):
    actual_rows = []

    if isinstance(data, dict):
        for key, value in data.items():
            if key == "Actual Rows":
                actual_rows.append(value)
            elif isinstance(value, (dict, list)):
                actual_rows.extend(extract_actual_rows(value))

    elif isinstance(data, list):
        for item in data:
            actual_rows.extend(extract_actual_rows(item))

    return actual_rows

In [98]:
df = pd.read_json('new.json')



In [99]:
columns =['Plan','query','avg_row_length','no_of_partitions','Execution Time']

In [100]:
df_copy = df[columns]


In [101]:
relation_names=[]
for row in df_copy['Plan']:
    row = json.dumps(row)
    # print(row)
    relation_name = extract_relation_names([json.loads(row)])
    relation_names.append(relation_name)

In [102]:
df_copy['relation_names'] = relation_names


C:\Users\shahb\AppData\Local\Temp\ipykernel_21288\2514779575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['relation_names'] = relation_names


In [103]:
filters=[]
for row in df_copy['Plan']:
    row = json.dumps(row)
    
    filter = extract_filters(json.loads(row))
    filters.append(filter)

In [104]:
df_copy['Plan']

0     {'Node Type': 'Limit', 'Parallel Aware': False...
1     {'Node Type': 'Limit', 'Parallel Aware': False...
2     {'Node Type': 'Limit', 'Parallel Aware': False...
3     {'Node Type': 'Limit', 'Parallel Aware': False...
4     {'Node Type': 'Limit', 'Parallel Aware': False...
5     {'Node Type': 'Limit', 'Parallel Aware': False...
6     {'Node Type': 'Limit', 'Parallel Aware': False...
7     {'Node Type': 'Limit', 'Parallel Aware': False...
8     {'Node Type': 'Limit', 'Parallel Aware': False...
9     {'Node Type': 'Limit', 'Parallel Aware': False...
10    {'Node Type': 'Limit', 'Parallel Aware': False...
11    {'Node Type': 'Limit', 'Parallel Aware': False...
12    {'Node Type': 'Limit', 'Parallel Aware': False...
13    {'Node Type': 'Limit', 'Parallel Aware': False...
14    {'Node Type': 'Limit', 'Parallel Aware': False...
15    {'Node Type': 'Limit', 'Parallel Aware': False...
16    {'Node Type': 'Limit', 'Parallel Aware': False...
17    {'Node Type': 'Limit', 'Parallel Aware': F

In [105]:
filter_len = []
for i in filters:
    filter_len.append(len(i))

df_copy['filters'] = filters
df_copy['filter_len'] = filter_len  

C:\Users\shahb\AppData\Local\Temp\ipykernel_21288\3572001993.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['filters'] = filters


In [106]:
cardinality=[]
for row in df_copy['Plan']:
    row = json.dumps(row)
    temp = extract_actual_rows(json.loads(row))
    cardinality.append(temp)


df_copy['cardinality'] = cardinality


In [107]:
# df_copy['output_cardinality'] = 
first_elements = df_copy['cardinality'].apply(lambda x: x[0])

# Creating a new DataFrame with the first elements
first_elements_df = pd.DataFrame(first_elements, columns=['first_element'])

# Displaying the new DataFrame
df_copy['output_cardinality']=first_elements

In [108]:
def extract_actual_rows_with_level(data, level):
    actual_rows = []

    if isinstance(data, dict):
        for key, value in data.items():
            if key == "Actual Rows":
                actual_rows.append((value, level))
            elif isinstance(value, (dict, list)):
                actual_rows.extend(extract_actual_rows_with_level(value, level + 1))

    elif isinstance(data, list):
        for item in data:
            actual_rows.extend(extract_actual_rows_with_level(item, level + 1))

    return actual_rows

In [109]:
cardinality_with_level = []
for row in df_copy['Plan']:
    row = json.dumps(row)
    temp = extract_actual_rows_with_level(json.loads(row), 0)
    cardinality_with_level.append(temp)

df_copy['cardinality_with_level'] = cardinality_with_level

In [110]:
max_length = df_copy['no_of_partitions'].apply(len).max()
print(max_length)

7


In [111]:
for i in range(max_length):
    df_copy[f'partition_{i+1}'] = df_copy['no_of_partitions'].apply(lambda x: x[i] if len(x) > i else 0)


In [112]:
df_copy.drop(columns=['no_of_partitions'], inplace=True)

In [113]:
max_length = df_copy['avg_row_length'].apply(len).max()
print(max_length)

7


In [114]:
for i in range(max_length):
    df_copy[f'row_length_{i+1}'] = df['avg_row_length'].apply(lambda x: x[i] if len(x) > i else 0)


In [115]:
df_copy.drop(columns=['avg_row_length'], inplace=True)

In [116]:
df_copy['number_of_relations'] = df_copy['relation_names'].apply(len)

In [117]:
df_copy.head(10)

,Plan,query,Execution Time,relation_names,filters,filter_len,cardinality,output_cardinality,cardinality_with_level,partition_1,...,partition_6,partition_7,row_length_1,row_length_2,row_length_3,row_length_4,row_length_5,row_length_6,row_length_7,number_of_relations
0,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",198436.894,{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1,"[1, 1, 4, 3, 4, 9786269]",1,"[(1, 0), (1, 2), (4, 4), (3, 6), (4, 8), (9786...",0,...,0,0,156.322575,0.000000,0.000000,0.000000,0.0,0.0,0.0,1
1,"{'Node Type': 'Limit', 'Parallel Aware': False...","\nselect\n\tc_custkey,\n\tc_name,\n\tsum(l_ext...",37083.637,"{customer, lineitem, orders, nation}",[(l_returnflag = 'R'::bpchar)],1,"[20, 20, 190271, 190918, 63639, 191035, 191035...",20,"[(20, 0), (20, 2), (190271, 4), (190918, 6), (...",0,...,0,0,141.494798,191.867068,139.160000,156.322575,0.0,0.0,0.0,4
2,"{'Node Type': 'Limit', 'Parallel Aware': False...","\n\nselect\n\tps_partkey,\n\tsum(ps_supplycost...",29310.312,"{supplier, partsupp, nation}","[(n_name = 'EGYPT'::bpchar), (sum((partsupp.ps...",3,"[0, 1, 2, 1, 80160, 1002, 25000, 1, 1, 80, 0, ...",0,"[(0, 0), (1, 2), (2, 4), (1, 6), (80160, 8), (...",0,...,0,0,178.073920,177.418246,139.160000,0.000000,0.0,0.0,0.0,3
3,"{'Node Type': 'Limit', 'Parallel Aware': False...","\n\nselect\n\tl_shipmode,\n\tsum(case\n\t\twhe...",26306.650,"{lineitem, orders}","[((l_shipmode = ANY ('{RAIL,AIR}'::bpchar[])) ...",1,"[1, 1, 78601, 78601, 26976, 52335, 1]",1,"[(1, 0), (1, 2), (78601, 4), (78601, 6), (2697...",0,...,0,0,141.494798,156.322575,0.000000,0.000000,0.0,0.0,0.0,2
4,"{'Node Type': 'Limit', 'Parallel Aware': False...","\nselect\n\tc_count,\n\tcount(*) as custdist\n...",8032.302,"{customer, orders}",[((o_comment)::text !~~ '%special%accounts%'::...,1,"[1, 1, 43, 750000, 750000, 250000, 2556553, 25...",1,"[(1, 0), (1, 2), (43, 4), (750000, 6), (750000...",0,...,0,0,141.494798,191.867068,0.000000,0.000000,0.0,0.0,0.0,2
5,"{'Node Type': 'Limit', 'Parallel Aware': False...",\nselect\n\t100.00 * sum(case\n\t\twhen p_type...,22582.863,"{lineitem, part}",[],0,"[1, 1, 3, 1, 128859, 128859, 386576, 333333, 3...",1,"[(1, 0), (1, 2), (3, 4), (1, 6), (128859, 8), ...",0,...,0,0,158.343460,156.322575,0.000000,0.000000,0.0,0.0,0.0,2
6,"{'Node Type': 'Limit', 'Parallel Aware': False...","\nselect\n\tp_brand,\n\tp_type,\n\tp_size,\n\t...",2364.004,"{supplier, partsupp, part}","[(NOT (hashed SubPlan 1)), ((s_comment)::text ...",3,"[1, 1, 27686, 590960, 196987, 196987, 1332640,...",1,"[(1, 0), (1, 2), (27686, 4), (590960, 6), (196...",0,...,0,0,177.418246,158.343460,178.073920,0.000000,0.0,0.0,0.0,3
7,"{'Node Type': 'Limit', 'Parallel Aware': False...",\nselect\n\tsum(l_extendedprice) / 7.0 as avg_...,77410.038,"{lineitem, part}",[((p_brand = 'Brand#51'::bpchar) AND (p_contai...,2,"[1, 1, 2674, 1024, 1024, 341, 998867, 2996450,...",1,"[(1, 0), (1, 2), (2674, 4), (1024, 6), (1024, ...",0,...,0,0,158.343460,156.322575,0.000000,0.000000,0.0,0.0,0.0,2
8,"{'Node Type': 'Limit', 'Parallel Aware': False...","\n\nselect\n\tc_name,\n\tc_custkey,\n\to_order...",28393.979,"{customer, lineitem, orders}",[(sum(lineitem_1.l_quantity) > '312'::numeric)],1,"[55, 55, 55, 55, 18, 128, 128, 18, 2500000, 25...",55,"[(55, 0), (55, 2), (55, 4), (55, 6), (18, 8), ...",0,...,0,0,141.494798,191.867068,156.322575,0.000000,0.0,0.0,0.0,3
9,"{'Node Type': 'Limit', 'Parallel Aware': False...",\n\nselect\n\tsum(l_extendedprice* (1 - l_disc...,4667.091,"{lineitem, part}",[((p_size >= 1) AND (((p_brand = 'Brand#33'::b...,2,"[1, 1, 3, 1, 188, 806, 0]",1,"[(1, 0), (1, 2), (3, 4), (1, 6), (188, 8), (80...",0,...,0,0,158.343460,156.322575,0.000000,0.000000,0.0,0.0,0.0,2


In [119]:
df_copy['cardinality_with_level']=df_copy['cardinality_with_level'].apply(lambda x: [(y[0],int(y[1]/2)) for y in x])

In [122]:
base_cardinality=[]
level_cardinality=[]
cardinality = df_copy['cardinality_with_level'][0]

In [123]:
cardinality

[(1, 0), (1, 1), (4, 2), (3, 3), (4, 4), (9786269, 5)]